In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv('Fake_data.csv')

# Display the first few rows of the dataframe
df.head()

In [ ]:
import spacy
import re
from spacy.tokens import Span

# Function to scan text file for sensitive data
def scan_text_file(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    # Define a regex pattern for phone numbers
    PHONE_PATTERN = r"\(\d{3}\) \d{3}-\d{4}"
    # Define a regex pattern for emails
    EMAIL_PATTERN = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
    # Define a regex pattern for employee IDs
    EMPLOYEE_ID_PATTERN = r"EMP-\d{6}"
    # Define a regex pattern for IP addresses
    IP_ADDRESS_PATTERN = r"\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b"
    doc = nlp(text)
    sensitive_data = []

       # Detect phone numbers
    phone_matches = re.finditer(PHONE_PATTERN, text)
    for match in phone_matches:
        phone_number = match.group()
        sensitive_data.append((phone_number, "PHONE_NUMBER"))
        print(phone_number, "PHONE_NUMBER")

    # Detect emails
    email_matches = re.finditer(EMAIL_PATTERN, text)
    for match in email_matches:
        email = match.group()
        sensitive_data.append((email, "EMAIL"))
        print(email, "EMAIL")
        # Detect employee IDs
    employee_id_matches = re.finditer(EMPLOYEE_ID_PATTERN, text)
    for match in employee_id_matches:
            employee_id = match.group()
            sensitive_data.append((employee_id, "EMPLOYEE_ID"))
            print(employee_id, "EMPLOYEE_ID")

        # Detect IP addresses
    ip_address_matches = re.finditer(IP_ADDRESS_PATTERN, text)
    for match in ip_address_matches:
            ip_address = match.group()
            sensitive_data.append((ip_address, "IP_ADDRESS"))
            print(ip_address, "IP_ADDRESS")

    # Detect other named entities
    for ent in doc.ents:
        print(ent.text, ent.label_)
        if ent.label_ in ["PERSON", "EMAIL", "GPE"]:
            sensitive_data.append((ent.text, ent.label_))
    
    return sensitive_data
# Load and scan the text file
file_path = 'train.txt'
sensitive_data = scan_text_file(file_path)
print(sensitive_data)

In [ ]:

# Load and scan the text file for sensitive data
file_path = 'train.txt'
sensitive_data = scan_text_file(file_path)

# Function to mask sensitive data
def mask_sensitive_data(text):
    for data, label in sensitive_data:
        text = text.replace(data, '*' * len(data))
    return text

# Read the file content
with open(file_path, 'r') as file:
    text = file.read()

# Mask sensitive data in the text
masked_text = mask_sensitive_data(text)

# Save the masked text to a new file
with open('checked.txt', 'w') as file:
    file.write(masked_text)

In [ ]:
import fitz  # PyMuPDF

# Function to extract text from specific pages of a PDF
def extract_text_from_pages(pdf_path, page_numbers):
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    extracted_text = ""

    # Iterate through the specified page numbers
    for page_num in page_numbers:
        # Load the page
        page = pdf_document.load_page(page_num - 1)  # page numbers are 0-based in PyMuPDF
        # Extract text from the page
        extracted_text += page.get_text()

    return extracted_text, pdf_document.page_count

# Specify the path to the PDF file and the pages to extract
pdf_path = 'again.pdf'
# Extract text from all pages and count the number of pages
text, num_pages = extract_text_from_pages(pdf_path, range(1, fitz.open(pdf_path).page_count + 1))
print(f"Number of pages in the PDF: {num_pages}")

# Remove white spaces and tabs from the extracted text
text = ' '.join(text.split())
print(text)

# Count the number of tokens in the text variable
doc = nlp(text)

num_tokens = len(doc)
print(f"Number of tokens in the text: {num_tokens}")


In [ ]:
import re
import spacy
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

# Load the NLP model
nlp = spacy.load("en_core_web_sm")

def mask_sensitive_data(text):
    # Define regex patterns
 
    PHONE_PATTERN = r"\+973 \d{8}"
    EMAIL_PATTERN = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
    EMPLOYEE_ID_PATTERN = r"EMP-\d{6}"
    IP_ADDRESS_PATTERN = r"\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b"
    CREDIT_CARD_PATTERN = r"\b(?:\d[ -]*?){13,16}\b"

    # Masking function
    def mask_match(match):
        return "*" * len(match.group())

    # Mask sensitive data using regex
    masked_text = re.sub(PHONE_PATTERN, mask_match, text)
    masked_text = re.sub(EMAIL_PATTERN, mask_match, masked_text)
    masked_text = re.sub(EMPLOYEE_ID_PATTERN, mask_match, masked_text)
    masked_text = re.sub(IP_ADDRESS_PATTERN, mask_match, masked_text)
    masked_text = re.sub(CREDIT_CARD_PATTERN, mask_match, masked_text)

    # Process with NLP for named entities
    doc = nlp(text)
    # Print all entity labels in the spaCy library
    for label in nlp.get_pipe("ner").labels:
        print(label)
    for ent in doc.ents:
      
        print(ent.text, ent.label_)
        if ent.label_ in ["PERSON", "EMAIL", "GPE","LOC","FAC"]:  # Mask names, emails, and locations
            print(ent.text, ent.label_)
            masked_text = masked_text.replace(ent.text, "*" * len(ent.text))

    return masked_text
def save_as_pdf(text, filename="masked_text.pdf"):
    # Create a PDF
    pdf = canvas.Canvas(filename, pagesize=letter)
    pdf.setFont("Helvetica", 12)

    # Split text into lines for PDF formatting
    lines = text.split("\n")
    y_position = 750  # Starting position on page

    for line in lines:
        # Split long lines to fit within the page width
        while len(line) > 90:
            pdf.drawString(50, y_position, line[:90])
            line = line[90:]
            y_position -= 20
            if y_position < 50:  
                pdf.showPage()
                pdf.setFont("Helvetica", 12)
                y_position = 750

        # Add new line after each period
        sentences = line.split('. ')
        for sentence in sentences:
            pdf.drawString(50, y_position, sentence.strip() + '.')
            y_position -= 20
            if y_position < 50:  
                pdf.showPage()
                pdf.setFont("Helvetica", 12)
                y_position = 750

        if y_position < 50:  
            pdf.showPage()
            pdf.setFont("Helvetica", 12)
            y_position = 750

    pdf.save()
    print(f"PDF saved as {filename}")


# Mask data and save it as a PDF
masked_text = mask_sensitive_data(text)
save_as_pdf(masked_text)

# Print masked text (optional)
print(masked_text)


In [26]:
def check_url_safety(url, api_key):
    # Encode the URL in base64 format (as required by VirusTotal)
    url_id = base64.urlsafe_b64encode(url.encode()).decode().strip("=")
    vt_url = f"https://www.virustotal.com/api/v3/urls/{url_id}"
    headers = {"x-apikey": api_key}
    
    try:
        response = requests.get(vt_url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            malicious_count = data.get("data", {}).get("attributes", {}).get("last_analysis_stats", {}).get("malicious", 0)
            return "Malicious" if malicious_count > 0 else "Safe"
        else:
            return f"Error: {response.status_code} {response.reason}"
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
import base64
import requests
def main():

   
    
        # Check for URLs
    urls ="http://testsafebrowsing.appspot.com/s/phishing.html"
    print(f"Found URL: {urls}")
    result = check_url_safety(urls, api_key)
    print(f"URL Safety Check Result: {result}")
      

        # Check for blacklisted words

if __name__ == "__main__":
    main()

Found URL: http://testsafebrowsing.appspot.com/s/phishing.html
URL Safety Check Result: Malicious


In [11]:
import os
import tenseal as ts
from cryptography.fernet import Fernet

KEYS_DIR = "ProcessedFiles/keys"
ENCRYPTED_DIR = "ProcessedFiles/encrypted"

# ----- AES-256 ENCRYPTION -----
def generate_key():
    """Generate and return an AES encryption key."""
    return Fernet.generate_key()

def save_key(key, file_name):
    """Save encryption key."""
    if not os.path.exists(KEYS_DIR):
        os.makedirs(KEYS_DIR)
    key_path = os.path.join(KEYS_DIR, file_name + ".key")
    with open(key_path, "wb") as kf:
        kf.write(key)

def encrypt_data(data, key):
    """Encrypt data using AES-256."""
    cipher = Fernet(key)
    return cipher.encrypt(data)

def decrypt_data(encrypted_data, key):
    """Decrypt data using AES-256."""
    cipher = Fernet(key)
    return cipher.decrypt(encrypted_data)

def save_encrypted_file(encrypted_data, file_name):
    """Save the encrypted file to disk."""
    if not os.path.exists(ENCRYPTED_DIR):
        os.makedirs(ENCRYPTED_DIR)
    file_path = os.path.join(ENCRYPTED_DIR, file_name)
    with open(file_path, "wb") as ef:
        ef.write(encrypted_data)

def read_encrypted_file(file_name):
    """Read and return the encrypted file contents."""
    file_path = os.path.join(ENCRYPTED_DIR, file_name)
    with open(file_path, "rb") as ef:
        return ef.read()

# ----- TEST FILE ENCRYPTION -----
if __name__ == "__main__":
    aes_key = generate_key()
    save_key(aes_key, "file_key")

    # Original Data
    data = b"Sensitive File Content: User ID 123456, SSN: 987-65-4321"

    # Encrypt and Save
    encrypted_file_data = encrypt_data(data, aes_key)
    save_encrypted_file(encrypted_file_data, "encrypted_file.dat")

    # Read and Display Encrypted Content
    encrypted_content = read_encrypted_file("encrypted_file.dat")
    print("\n🔒 Encrypted File Content (Hex):", encrypted_content.hex())

    # Decrypt and Display Original Content
    decrypted_file_data = decrypt_data(encrypted_content, aes_key)
    print("\n🔓 Decrypted File Content:", decrypted_file_data.decode())



🔒 Encrypted File Content (Hex): 674141414141426e7361727966677337796e4a47562d4943584634724f73775a6a4c777771346e6b534155714c633536333578717a43684f7a2d577963425037784239414a53454b4d5635656e354a586f4b63682d375f546a773448303567546b4852773674674e7949795543645f487371624a51424e4f6f6d786a49475671547267546a4a36304337323057506d754a7766686e70714a5f2d45747842456e6e773d3d

🔓 Decrypted File Content: Sensitive File Content: User ID 123456, SSN: 987-65-4321


In [1]:
TRAIN_DATA = [
    ("My Bahraini number is +973 3345 6789.", {"entities": [(21, 35, "PHONE_NUMBER")]}),
    ("You can reach me at +973 1245 6789.", {"entities": [(19, 33, "PHONE_NUMBER")]}),
    ("Call me on +973 9988 1122.", {"entities": [(15, 29, "PHONE_NUMBER")]}),
    ("My contact number is +973 3XXX XXXX.", {"entities": [(24, 38, "PHONE_NUMBER")]}),
    ("For emergencies, call +973 8000 9999.", {"entities": [(30, 44, "PHONE_NUMBER")]}),
]


In [ ]:
import spacy
from spacy.training.example import Example
from spacy.training import offsets_to_biluo_tags

# Load the pre-trained model (you can use a smaller one like 'en_core_web_sm')
nlp = spacy.load("en_core_web_lg")

# Add a new entity label for phone numbers if it's not already in the pipeline
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
ner = nlp.get_pipe("ner")
ner.add_label("PHONE_NUMBER")

# Prepare the optimizer and begin training
optimizer = nlp.resume_training()

# Fine-tune the model on the provided training data
for epoch in range(100):
    print(f"Epoch: {epoch + 1}")
    for text, annotations in TRAIN_DATA:
        doc = nlp.make_doc(text)
        tags = offsets_to_biluo_tags(doc, annotations["entities"])
        example = Example.from_dict(doc, {"entities": annotations["entities"], "tags": tags})
        example = Example.from_dict(doc, {"entities": annotations["entities"]})
        nlp.update([example], sgd=optimizer)

# Save the fine-tuned model
nlp.to_disk("bahraini_phone_number_model")


c:\Users\xxlm5\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "My Bahraini number is +973 3345 6789." with entities "[(21, 35, 'PHONE_NUMBER')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\xxlm5\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "You can reach me at +973 1245 6789." with entities "[(19, 33, 'PHONE_NUMBER')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\xxlm5\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities co

In [7]:
# Additional training data
MORE_TRAIN_DATA = [
    ("Contact us at +973 4455 6677.", {"entities": [(14, 28, "PHONE_NUMBER")]}),
    ("Emergency number: +973 5566 7788.", {"entities": [(18, 32, "PHONE_NUMBER")]}),
    ("Reach out at +973 6677 8899.", {"entities": [(13, 27, "PHONE_NUMBER")]}),
    ("Call +973 7788 9900 for support.", {"entities": [(5, 19, "PHONE_NUMBER")]}),
    ("Phone: +973 8899 0011.", {"entities": [(7, 21, "PHONE_NUMBER")]}),
]

# Combine the original and additional training data
TRAIN_DATA.extend(MORE_TRAIN_DATA)

# Fine-tune the model on the combined training data
for epoch in range(200):  # Increase the number of epochs
    print(f"Epoch: {epoch + 1}")
    for text, annotations in TRAIN_DATA:
        doc = nlp.make_doc(text)
        tags = offsets_to_biluo_tags(doc, annotations["entities"])
        example = Example.from_dict(doc, {"entities": annotations["entities"], "tags": tags})
        example = Example.from_dict(doc, {"entities": annotations["entities"]})
        nlp.update([example], sgd=optimizer)

# Save the fine-tuned model
nlp.to_disk("bahraini_phone_number_model")

# Load the fine-tuned model
nlp = spacy.load("bahraini_phone_number_model")

# Test the model on a new sentence
doc = nlp("For assistance, contact +973 3345 6789 or +973 8000 5555.")
for ent in doc.ents:
    print(f"Detected entity: {ent.text} with label: {ent.label_}")
    if ent.label_ == "PHONE_NUMBER":
        print(f"Phone number detected: {ent.text}")


Epoch: 1


c:\Users\xxlm5\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "My Bahraini number is +973 3345 6789." with entities "[(21, 35, 'PHONE_NUMBER')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\xxlm5\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "You can reach me at +973 1245 6789." with entities "[(19, 33, 'PHONE_NUMBER')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\xxlm5\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities co

Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9
Epoch: 10
Epoch: 11
Epoch: 12
Epoch: 13
Epoch: 14
Epoch: 15
Epoch: 16
Epoch: 17
Epoch: 18
Epoch: 19
Epoch: 20
Epoch: 21
Epoch: 22
Epoch: 23
Epoch: 24
Epoch: 25
Epoch: 26
Epoch: 27
Epoch: 28
Epoch: 29
Epoch: 30
Epoch: 31
Epoch: 32
Epoch: 33
Epoch: 34
Epoch: 35
Epoch: 36
Epoch: 37
Epoch: 38
Epoch: 39
Epoch: 40
Epoch: 41
Epoch: 42
Epoch: 43
Epoch: 44
Epoch: 45
Epoch: 46
Epoch: 47
Epoch: 48
Epoch: 49
Epoch: 50
Epoch: 51
Epoch: 52
Epoch: 53
Epoch: 54
Epoch: 55
Epoch: 56
Epoch: 57
Epoch: 58
Epoch: 59
Epoch: 60
Epoch: 61
Epoch: 62
Epoch: 63
Epoch: 64
Epoch: 65
Epoch: 66
Epoch: 67
Epoch: 68
Epoch: 69
Epoch: 70
Epoch: 71
Epoch: 72
Epoch: 73
Epoch: 74
Epoch: 75
Epoch: 76
Epoch: 77
Epoch: 78
Epoch: 79
Epoch: 80
Epoch: 81
Epoch: 82
Epoch: 83
Epoch: 84
Epoch: 85
Epoch: 86
Epoch: 87
Epoch: 88
Epoch: 89
Epoch: 90
Epoch: 91
Epoch: 92
Epoch: 93
Epoch: 94
Epoch: 95
Epoch: 96
Epoch: 97
Epoch: 98
Epoch: 99
Epoch: 100
Epoch: 101
Epoch: